In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import VarianceThreshold, mutual_info_regression
from sklearn.tree import DecisionTreeRegressor

# 读取Excel文件，指定header=0来使用第一行作为列名
df = pd.read_excel("D:\\张雨林\\研三工作\\力学工站制备高通量_formulations_whole.xlsx", 
                   sheet_name="Sheet2", 
                   header=0)

# 打印列名以验证
print("Columns:")
print(df.columns)

# 打印前几行数据以检查
print("\nFirst few rows:")
print(df.head())

# 将所有列转换为数值类型，非数值的转换为NaN
df = df.apply(pd.to_numeric, errors='coerce')

# 将所有NaN值替换为0
df = df.fillna(0)

# 分离特征和目标变量
X = df.iloc[:, :-1]  # 所有列除了最后一列
y = df.iloc[:, -1]   # 最后一列（断裂能）

print(f"\nInitial number of features: {X.shape[1]}")

# 2. 自相关性分析（方差阈值）
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

var_thresh = VarianceThreshold(threshold=0.01)  # 可以调整阈值
X_var_thresholded = var_thresh.fit_transform(X_scaled)

print(f"Features after variance thresholding: {X_var_thresholded.shape[1]}")

# 获取方差阈值筛选后的特征名称和数值
var_thresholded_features = X.columns[var_thresh.get_support()].tolist()
var_thresholded_values = X_var_thresholded

# 3. 互相关性分析（基于互信息）
mi_scores = mutual_info_regression(X_var_thresholded, y)
mi_scores = pd.Series(mi_scores, index=X.columns[var_thresh.get_support()])
mi_scores = mi_scores.sort_values(ascending=False)

# 选择互信息分数最高的前100个特征
top_100_features = mi_scores.nlargest(100).index.tolist()
X_mi_selected = X_var_thresholded[:, [list(X.columns[var_thresh.get_support()]).index(col) for col in top_100_features]]

print(f"Features after mutual information selection: {X_mi_selected.shape[1]}")

# 获取互信息筛选后的特征名称和数值
mi_selected_features = top_100_features
mi_selected_values = X_mi_selected

# 4. 决策树特征重要性
dt = DecisionTreeRegressor(random_state=42)
dt.fit(X_mi_selected, y)

feature_importance = pd.Series(dt.feature_importances_, index=top_100_features).sort_values(ascending=False)

# 选择重要性大于平均值的特征
important_features = feature_importance[feature_importance > feature_importance.mean()].index.tolist()
X_final = X_mi_selected[:, [top_100_features.index(col) for col in important_features]]

print(f"Final number of features: {X_final.shape[1]}")

# 获取决策树筛选后的特征名称和数值
dt_selected_features = important_features
dt_selected_values = X_final

# 创建Excel writer对象
with pd.ExcelWriter('feature_selection_results.xlsx') as writer:
    # 保存方差阈值筛选结果到Sheet1
    pd.DataFrame(var_thresholded_values, columns=var_thresholded_features).to_excel(writer, sheet_name='Variance_Thresholded', index=False)
    
    # 保存互信息筛选结果到Sheet2
    pd.DataFrame(mi_selected_values, columns=mi_selected_features).to_excel(writer, sheet_name='Mutual_Information_Selected', index=False)
    
    # 保存决策树筛选结果到Sheet3
    pd.DataFrame(dt_selected_values, columns=dt_selected_features).to_excel(writer, sheet_name='Decision_Tree_Selected_B', index=False)

print("Results have been saved to 'feature_selection_results.xlsx'")

# 输出最终选择的特征
print("Final selected features:")
print(important_features)

